<a href="https://colab.research.google.com/github/Z3376/Semantic-Text-Similarity-with-BERT-and-USE/blob/master/USE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from time import time

# Data

In [0]:
!wget "https://storage.googleapis.com/kaggle-competitions-data/kaggle/6277/323734/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1564850519&Signature=ZOmgoiCiwnco7UWfkhGupcrP2HYfacbojgKaN5orkSW%2BJ%2BnEi5vgtmEqdFZ3o5KIkIIotnFSntXvK0Ks21oIx1OSOhjDYrhh3FxUD34nbW6KohudGln%2B63QXtFoQmuXIJlRn4xdo9f3y0Jwx7Y13I0b6wzXt4fTKMz8s83DSo%2FXSwtCLS4nZhd1nPDT0TcFkjuxl%2B60CBwS0jSSYPVXbFzJG30qDqyvhWKeKGuOg88LrXA%2BTQNIzc1bYDBRIPWBRffhHMpHYIRdfDGtAZb5pq9w0qn4Fx8XG9M1zI1pgfwH7u8v30uYnR0lrt16%2BWaURoKGJJ9ex0qLnoF9Y8HBWMw%3D%3D&response-content-disposition=attachment%3B+filename%3Dquora-question-pairs.zip"

In [0]:
!unzip "all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1564850519&Signature=ZOmgoiCiwnco7UWfkhGupcrP2HYfacbojgKaN5orkSW+J+nEi5vgtmEqdFZ3o5KIkIIotnFSntXvK0Ks21oIx1OSOhjDYrhh3FxUD34nbW6KohudGln+63QXtFoQmuXIJlRn4xdo9"
!rm -f "all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1564850519&Signature=ZOmgoiCiwnco7UWfkhGupcrP2HYfacbojgKaN5orkSW+J+nEi5vgtmEqdFZ3o5KIkIIotnFSntXvK0Ks21oIx1OSOhjDYrhh3FxUD34nbW6KohudGln+63QXtFoQmuXIJlRn4xdo9"

# TF_Hub Module

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

In [0]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder-lite/2"
# embed_lite = hub.Module(module_url)
# with tf.Session as sess:
#     spm_path = sess.run(module(signature="spm_path"))
# sp = spm.SentencePieceProcessor()
# sp.Load(spm_path)

# Model

In [0]:
def USE(x):
    return embed(tf.squeeze(tf.cast(x,tf.string)),signature="default",as_dict=True)["default"]

In [0]:
def USE_lite(x):
    return embed_lite(tf.squeeze(tf.cast(x,tf.string)),signature="default",as_dict=True)["default"]

In [0]:
def add(x):
    return tf.reduce_sum(x,keepdims=True,axis=0)

In [0]:
def reshape(x):
    return tf.reshape(x,(512,1))

In [0]:
def cosine_similarity(x):
    z = x[2]
    y = x[1]
    x = x[0]
    return x/keras.backend.dot(keras.backend.sqrt(y),keras.backend.sqrt(z))

In [0]:
# inpt = keras.layers.Input((1,),dtype=tf.string)
# embedding = keras.layers.Lambda(USE,output_shape=(512,))
# outpt = embedding(inpt)
# outpt0 = keras.layers.Lambda(reshape)(outpt[0])
# outpt1 = keras.layers.Lambda(reshape)(outpt[1])
# dot01 = keras.layers.Multiply()([outpt0,outpt1])
# dot01 = keras.layers.Lambda(add)(dot01)
# dot00 = keras.layers.Multiply()([outpt0,outpt0])
# dot00 = keras.layers.Lambda(add)(dot00)
# dot11 = keras.layers.Multiply()([outpt1,outpt1])
# dot11 = keras.layers.Lambda(add)(dot11)
# similarity = keras.layers.Lambda(cosine_similarity)([dot01,dot00,dot11])
# model = keras.models.Model(inpt,similarity)

In [0]:
inpt = keras.layers.Input((1,),dtype=tf.string)
embedding = keras.layers.Lambda(USE,output_shape=(512,))
outpt = embedding(inpt)
model = keras.models.Model(inpt,outpt)

In [0]:
# inpt = keras.layers.Input((1,),dtype=tf.string)
# embedding = keras.layers.Lambda(USE_lite,output_shape=(512,))
# outpt = embedding(inpt)
# model_lite = keras.models.Model(inpt1,outpt)

In [0]:
model.summary()

In [0]:
# model.compile(optimizer='adam',loss='MSE',metrics=['accuracy'])

In [0]:
keras.utils.plot_model(model,'USEmodel.png')

# Inference

In [0]:
def mod(a):
    return np.sqrt(np.dot(a,a))

In [0]:
def similarity(pred,a,b):
    A = pred[a]
    B = pred[b]
    AB = np.dot(A,B)
    A_mod = mod(A)
    B_mod = mod(B)
    return AB/(A_mod*B_mod)

## dummy

In [0]:
sen = np.array(['hi there','hello','my phone is not working','what is happening?',"I don't know what's going on",'Time flies like an arrow','Fruit flies like a banana'])

In [0]:
with tf.Session() as sess:
    keras.backend.set_session(sess)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    pred = model.predict(sen)

In [0]:
l = len(pred)
sim = [[similarity(pred,i,j) for j in range(l)] for i in range(l)]
for i in range(l):
    print(['%.2f'%sim[i][j] for j in range(len(sim[i]))])

## Quora Q pairs

In [0]:
df = pd.read_csv('train.csv',encoding='utf-8')
que = df.iloc[:3*len(df)//4,-3:].values.astype('U')
que = np.array(sorted(que,key=lambda a_entry: a_entry[-1]))
y = [int(que[i,-1]) for i in range(len(que))]
que = que[:,:-1].flatten()

In [0]:
t0 = time()
with tf.Session() as sess1:
    keras.backend.set_session(sess1)
    sess1.run(tf.global_variables_initializer())
    sess1.run(tf.tables_initializer())
    pred = model.predict(que)
t1 = time()
print('Time Elapsed: ',t1-t0,'s')

In [0]:
sim = []
for i in range(len(que)//2):
    sim.append(similarity(pred,2*i,2*i+1))
sim = np.array(sim)

In [0]:
plt.figure()
plt.plot(sim)
plt.plot(y)
plt.show()

In [0]:
threshold = [0.25,0.4,0.5,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.925,0.95,0.975]
acc = []
precision_0 = []
precision_1 = []
recall_0 = []
recall_1 = []
for thresh in threshold:
    sim_t = [1 if sim[i]>thresh else 0 for i in range(len(sim))]
    cm = confusion_matrix(y,sim_t)
    acc.append((cm[0][0]+cm[1][1])/(len(que)//2))
    precision_0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
    precision_1.append(cm[1][1]/(cm[1][0]+cm[1][1]))
    recall_0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
    recall_1.append(cm[1][1]/(cm[1][1]+cm[0][1]))

In [0]:
plt.figure()
plt.title('Acc')
plt.plot(threshold,acc)
plt.grid()
plt.figure()
plt.title('Precision')
plt.plot(threshold,precision_0)
plt.plot(threshold,precision_1)
plt.legend(['prec_0','prec_1'])
plt.grid()
plt.figure()
plt.title('Recall')
plt.plot(threshold,recall_0)
plt.plot(threshold,recall_1)
plt.legend(['rec_0','rec_1'])
plt.grid()
plt.show()

In [0]:
np.max(acc)

In [0]:
sim = np.array(sim,dtype='float64')

In [0]:
score = log_loss(y,np.abs(sim))
print(score)

In [0]:
test = pd.read_csv('test.csv',encoding='utf-8')

In [0]:
len(test)

In [0]:
len(que)